In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Read CSV input

In [2]:
# Replace 'your_file.csv' with the path to your CSV file
file_path = 'sp500-companies.csv'

encoding = 'latin-1'  # Change this to the appropriate encoding

# Read the CSV file into a DataFrame with the specified encoding
df = pd.read_csv(file_path, encoding=encoding)

Convert CSV column Tickers into list

In [3]:
# Extract all values from 'Column A' and store them in a list
companies = df['Ticker'].tolist()

In [4]:
len(companies)

503

Read Buy/Sale Data

In [5]:
base_url = "http://openinsider.com/screener?s="

end_url = "&o=&pl=&ph=&ll=&lh=&fd=-1&fdr=01%2F01%2F2012+-+11%2F09%2F2023&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=1000&page=1"
# create user agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\
    37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"

In [17]:
def get_page(ticker, count=0):
    url = base_url + ticker + end_url
    # Some pages don't work without headers. See robots.txt
    page_ = requests.get(url, headers={"User-Agent": user_agent})

    if not page_.ok:  # Only requests 2XX are valid
        if count >= 5:
            raise Exception(f"Could not get page {self.url}")
        sleep(randint(1, 5))
        get_page(count+1)

    return page_

In [18]:
def get_soup(page):
    soup_ = BeautifulSoup(page.text, features="html.parser")
    return soup_

In [46]:
def get_rows(soup):
    try:
        rows = soup.find('table', {'class': 'tinytable'}).find('tbody').findAll('tr')
    except:
        # Rare case when an Ticker has no information
        return
    return rows

In [38]:
def clean_rows(rows, ticker_entry):
    records = []
    for val in rows:
        cols = val.findAll('td')
        if not cols:
            continue
        insider_data = {key: cols[index].find('a').text.strip() if cols[index].find('a') else cols[index].text.strip() 
                        for index, key in enumerate(['transaction_date', 'trade_date', 'ticker', 'company_name', 
                                                     'owner_name', 'Title', 'transaction_type', 'last_price', 'Qty', 
                                                     'shares_held', 'Owned', 'Value'])}
        records.append(tuple(insider_data.values()))
    return records

In [49]:
openinsider = []
counter = 0
for company in companies:
    page = get_page(company)
    soup = get_soup(page)
    rows = get_rows(soup)
    if rows is not None:
        records = clean_rows(rows, company)
        openinsider += records
        counter+=1
        if counter % 50 == 0:
            print(counter) # For time managemente

50
100
150
200
250
300
350
400
450


In [50]:
len(openinsider)

107835

In [53]:
# Define column names
columns = ['transaction_date', 'trade_date', 'ticker', 'company_name', 'owner_name', 'Title', 'transaction_type', 'last_price', 'Qty', 'shares_held', 'Owned', 'Value']

# Create DataFrame
df_output = pd.DataFrame(openinsider, columns=columns)

In [54]:
df_output

,transaction_date,trade_date,ticker,company_name,owner_name,Title,transaction_type,last_price,Qty,shares_held,Owned,Value
0,,2023-05-19 12:13:32,2023-05-18,MMM,Ashish Khandpur K,Group Pres,S - Sale,$99.27,"-9,432","22,620",-29%,"-$936,306"
1,,2023-05-01 10:36:07,2023-04-28,MMM,Hammes Eric D.,"EVP, Chief Count Gov Svc Off",S - Sale,$105.33,"-6,487","10,430",-38%,"-$683,243"
2,D,2022-10-31 09:22:30,2022-10-28,MMM,Rhodes Kevin H,"EVP, GC",S - Sale+OE,$126.40,"-5,703","2,033",-74%,"-$720,842"
3,D,2022-10-31 09:19:51,2022-10-28,MMM,Chavez Rodriguez Beatriz Karina,"SVP, Chief Strategy Officer",S - Sale+OE,$126.28,"-1,071","5,522",-16%,"-$135,246"
4,D,2022-08-18 10:00:16,2022-08-17,MMM,Hammes Eric D.,"EVP, Chief Count Gov Svc Off",S - Sale+OE,$147.31,"-3,145","8,818",-26%,"-$463,294"
...,...,...,...,...,...,...,...,...,...,...,...,...
107830,,2013-02-08 16:20:03,2013-02-06,ZTS,Knupp Catherine A.,EVP,P - Purchase,$26.00,"+1,000","1,000",New,"+$26,000"
107831,D,2013-02-08 16:19:51,2013-02-06,ZTS,Pfizer Inc,"Dir, 10%",S - Sale+OE,$0.00,"-99,015,000",0,-100%,-$0
107832,,2013-02-08 16:19:29,2013-02-06,ZTS,Chen Heidi C.,EVP,P - Purchase,$26.00,"+2,500","2,500",New,"+$65,000"
107833,,2013-02-08 16:19:06,2013-02-06,ZTS,Beaty Sandra J.,EVP,P - Purchase,$26.00,"+2,000","2,000",New,"+$52,000"


In [55]:
df_output.to_csv('open_insider.csv', index=False)